In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm, datasets
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import collections
from sklearn import svm
from sklearn.model_selection import KFold 
from sklearn.model_selection import RepeatedKFold 
from sklearn import metrics 
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import KFold 
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
import shap
from sklearn.ensemble import RandomForestClassifier

In [ ]:
TCL=pd.read_excel('../../Documents/Full_List_Of_Features.xls')
#Read in data

#Do any data formatting here (i.e. remap strings as ints; drop unescessary columns)

FeatureNames = list(TCL)
FeatureNames=FeatureNames[2:]
#Grab names for later

#I like working in np
TCLNP=TCL.to_numpy()

#Outcomes
y=TCLNP[:,1]
#Features
X=TCLNP[:,2:]

In [ ]:
#Lasso regression for feature selection with cross-validation 

from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
#Lasso needs your variables to be normalized
scaler = StandardScaler().fit(X_train) 
X_train_N = scaler.transform(X_train)
X_test_N = scaler.transform(X_test)


lasso = LassoCV(cv=5, random_state=0, max_iter=10000)
lasso.fit(X_train_N, y_train)
print(lasso.alpha_)

plt.semilogx(lasso.alphas_, lasso.mse_path_, ":")
plt.plot(
    lasso.alphas_ ,
    lasso.mse_path_.mean(axis=-1),
    "k",
    label="Average across the folds",
    linewidth=2,
)
plt.axvline(
    lasso.alpha_, linestyle="--", color="k", label="alpha: CV estimate"
)
print(lasso.alpha_) #THIS IS THE VALUE YOU NEED
plt.legend()
plt.xlabel("alphas")
plt.ylabel("Mean square error")
plt.title("Mean square error on each fold")
plt.axis("tight")
#plt.savefig("ClinicalML-Predictions/Lasso_Regularlization_V1.pdf", bbox_inches='tight', dpi=800)

In [ ]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectFromModel
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Lasso
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
scaler = StandardScaler()
scaler.fit(X_train)
sel_ = SelectFromModel(
    #C value below is inverse (^-1) of alpha value determined by 5x CV above)
    LogisticRegression(C=0.822,penalty='l1', solver='liblinear'))
sel_.fit(scaler.transform(X_train), y_train)
arr=list(sel_.get_support())
c=arr.count(True)
print(c) #Prints total number of chosen features
c=np.array(list(zip(arr, FeatureNames)))
print(c) #This will print all features, with true next to the chosen ones.

In [ ]:
#remove the unchosen features
for idx, decision in enumerate(c):
    if decision[0]=='False':
        TCL = TCL.drop(columns=[FeatureNames[idx]])
#display(TCL)
FeatureNames = list(TCL)
FeatureNames=FeatureNames[2:]
#regrab feature names

#back to np
TCLNP=TCL.to_numpy()


y=TCLNP[:,1]
X=TCLNP[:,2:]

In [ ]:
#DO NOT RUN THIS CELL 
#Manual Drop cell for any features removed by the SHAP dendrogram plot 
#TCL = TCL.drop(columns=['LAVI-Pre'])
#TCL = TCL.drop(columns=['SA_PostNoVein'])
#TCL = TCL.drop(columns=['Pre-A Fib Atrial Floor'])
#TCLNP=TCL.to_numpy()

#y=TCLNP[:,1]
#X=TCLNP[:,2:]

In [ ]:
#Classifier
#5-fold cross validation to ensure ~70/30 T/T split
#Stratified to ensure equal distrubution of +/- samples 
cv = StratifiedKFold(n_splits=5, shuffle=True)
#Random Forest
classifier = RandomForestClassifier(bootstrap = True, max_features = 'auto')
#Initializations
tprs = []
aucs = []
precisions =[]
recalls=[]
thresholdss=[]
y_real = []
y_proba = []
list_shap_values = list()
list_test_sets = list()
mean_precision = np.linspace(0,1,100)
mean_fpr = np.linspace(0, 1, 100)
plt.figure(figsize=(10,10))
i = 0
for train, test in cv.split(X, y):
    probas_ = classifier.fit(X[train], y[train]).predict_proba(X[test])
    # Compute ROC AUC curve and PR Curve  
    fpr, tpr, thresholds = metrics.roc_curve(y[test], probas_[:, 1], pos_label=1.0)
    tprs.append(np.interp(mean_fpr, fpr, tpr))
    precision, recall, thresholds = precision_recall_curve(y[test], probas_[:, 1], pos_label=1.0)
    precisions.append(precision)
    thresholdss.append(thresholds)
    recalls.append(recall)
    tprs[-1][0] = 0.0
    roc_auc = metrics.auc(fpr, tpr)
    aucs.append(roc_auc)
    #explainer=shap.KernelExplainer(model=classifier.predict_proba, data=X[train], link="logit")
    #Use tree explainer, I use model output=raw, but can be changed to logit. 
    explainer = shap.TreeExplainer(classifier, data=X[train].astype(float), feature_perturbation="interventional", model_outut="raw")
    #I had check additivity to be false because I found that it was acting poorly b/c of a low threshold value and 
    #my small sample size. Should turn this back on to make sure everything is working properly.
    shap_values=explainer.shap_values(X=X[test], check_additivity=False )
    #for each iteration we save the test_set index and the shap_values
    list_shap_values.append(shap_values)#####
    list_test_sets.append(test)######
    # Also save the real outcome and predicted outcome for sanity
    y_real.append(y[test])
    y_proba.append(probas_[:, 1])
    #plotting stuff per CV
    plt.plot(fpr, tpr, lw=1, alpha=0.3,
             label='ROC fold %d (AUC = %0.2f)' % (i, roc_auc))

    i += 1
#plotting stuff overall
plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
         label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = metrics.auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
plt.plot(mean_fpr, mean_tpr, color='b',
         label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
         lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                label=r'$\pm$ 1 std. dev.')

plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.xlabel('False Positive Rate',fontsize=18)
plt.ylabel('True Positive Rate',fontsize=18)
plt.title('Cross-Validation ROC of Random Forest',fontsize=18)
plt.legend(loc="lower right", prop={'size': 15})
plt.show()
#outputs ROC curve 

In [ ]:
#This plots PR curve. 

plt.figure(figsize=(10,10))
from sklearn.metrics import average_precision_score
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import auc
precision_array=[]
recall_array = np.linspace(0, 1, 100)
mean_recall = np.linspace(0, 1, 100)
reversed_mean_precision=0.0


for i in range(0, 5):
    precision_fold, recall_fold = precisions[i][::-1], recalls[i][::-1]  # reverse order of results
    prec_array = np.interp(recall_array, recall_fold, precision_fold)
    pr_auc = auc(recall_array, prec_array)
    precision_array.append(prec_array)

    reversed_mean_precision += np.interp(mean_recall, recall_fold, precision_fold)
    reversed_mean_precision[0] = 0.0
    
    lab_fold = 'PR fold %d AUPR=%.4f' % (i+1, pr_auc)
    plt.plot(recall_fold, precision_fold, alpha=0.3, label=lab_fold)
    
reversed_mean_precision /= 5
reversed_mean_precision[0] = 1
mean_auc_pr = auc(mean_recall, reversed_mean_precision)
    

mean_precision = np.mean(precision_array, axis=0)
std_precision = np.std(precision_array, axis=0)
plt.fill_between(recall_array, mean_precision + std_precision, mean_precision - std_precision, alpha=0.3, linewidth=0, color='grey')
#plt.title("PR curves; {} folds".format(k_fold.n_splits), weight="bold", fontsize=15)
plt.plot(mean_recall, ([reversed_mean_precision])[0],
         label='Mean PR (AUC = %0.2f  $\pm$ %0.2f)' % (mean_auc_pr, mean(std_precision)), lw=2, color='blue')  
plt.xlabel("Recall (Sensitivity)", fontsize=18)
plt.ylabel("Precision (PPV)", fontsize=18)
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])

plt.legend(loc="lower right", prop={'size': 18})
plt.title("PR Curve", fontsize=18)


In [ ]:
#This will plot your dendogram to assess feature redundancy via shap. See https://shap.readthedocs.io/en/latest/example_notebooks/overviews/Be%20careful%20when%20interpreting%20predictive%20models%20in%20search%20of%20causal%C2%A0insights.html
#Cutoff can be changed via clustering_cutoff
#Sometimes the implementation of shap_values is inconsistent, (this is somwhat documented). I manually mutate the 
#shap values here to get it into a useable format. 
explainer = shap.Explainer(classifier)
shap_values2 = explainer(X)
shap_values2.values=shap_values[0]
clust = shap.utils.hclust(X, y, linkage="single")
shap.plots.bar(shap_values2, clustering_cutoff=1, clustering=clust,  show=True)
#plt.savefig("V1F/Barplot.pdf", bbox_inches='tight', dpi=800)


In [ ]:
#Compile SHAP values 
#Here I format the variable shap_values as well. If shap_values[1] in the instances below doesnt work, you can try 
#the variable shap_values2 described above. 
shap.initjs()
test_set = list_test_sets[0]
shap_values = np.array(list_shap_values[0])
for i in range(1,len(list_test_sets)):
    test_set = np.concatenate((test_set,list_test_sets[i]),axis=0)
    shap_values = np.concatenate((shap_values,np.array(list_shap_values[i])),axis=1)
X_test = X[test_set]


In [1]:
#Various means to vizualize data -- run each in a seperate cell.  
#Beeswarm
shap.summary_plot(shap_values[1], X_test, feature_names=FeatureNamesHoldout, max_display=30, show=False)
#Dependence plot, where first value is index of feature you want to see, here is 5. 
shap.dependence_plot(5, shap_values[1], X_test, show=False, feature_names=FeatureNamesHoldout, interaction_index=9)#, xmin=50000, xmax=250000)
#Force plot where "6" is the index of the case you want to see within your testing set. 
shap.force_plot(explainer.expected_value[1], shap_values[1][6,:], X_test[6,:],feature_names=FeatureNames, plot_cmap=['#FFD300','#A1045A'], matplotlib=False, link="identity")
#waterfall plot (not used in paper)
shap.plots._waterfall.waterfall_legacy(explainer.expected_value[1], shap_values[0][1], X[test][1])
#Bar plot of mean importance
shap.summary_plot(shap_values[1], X_test, plot_type="bar", feature_names=FeatureNamesHoldout, max_display=30, show=False)


NameError: name 'shap' is not defined

In [ ]:
#Make sure to save everything
import dill                            #pip install dill --user
filename = 'globalsave_V1F-2.pkl'
dill.dump_session(filename)

In [ ]:
# and to load the session again:

import dill
filename = 'globalsave_V1F-2.pkl'

dill.load_session(filename)